# Object Relational Tutorial
https://docs.sqlalchemy.org/en/13/orm/tutorial.html  
https://docs.sqlalchemy.org/en/13/index.html  

In [1]:
%pylab inline

from pprint import pprint

Populating the interactive namespace from numpy and matplotlib


## Version check

In [2]:
import sqlalchemy
sqlalchemy.__version__

'1.3.3'

## Connecting

In [3]:
from sqlalchemy import create_engine
# engine = create_engine('sqlite:///:memory:', echo = True)
engine = create_engine('sqlite:///:memory:', echo = False)
engine

Engine(sqlite:///:memory:)

## Declare a Mapping

In [4]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
Base

sqlalchemy.ext.declarative.api.Base

In [5]:
from sqlalchemy import Column, Integer, String, Sequence

class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, Sequence('user_id_seq'), primary_key=True)
    name = Column(String(50))
    fullname = Column(String(50))
    nickname = Column(String(50))
    
    def __repr__(self):
        return '<User(name = {}, fullname = {}, nickname = {})>'.format(self.name, self.fullname, self.nickname)

## Create a Schema

In [6]:
User.__table__

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False, default=Sequence('user_id_seq', metadata=MetaData(bind=None))), Column('name', String(length=50), table=<users>), Column('fullname', String(length=50), table=<users>), Column('nickname', String(length=50), table=<users>), schema=None)

In [7]:
Base.metadata

MetaData(bind=None)

In [8]:
Base.metadata.create_all(engine)

## Create an Instance of the Mapped Class

In [9]:
ed_user = User(name = 'ed', fullname = 'Ed Jones', nickname = 'edsnickname')
ed_user

<User(name = ed, fullname = Ed Jones, nickname = edsnickname)>

In [10]:
ed_user.name

'ed'

In [11]:
str(ed_user.id)

'None'

## Creating a Session

In [12]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind = engine)
Session

sessionmaker(class_='Session', bind=Engine(sqlite:///:memory:), autoflush=True, autocommit=False, expire_on_commit=True)

In [13]:
Session = sessionmaker()
Session.configure(bind = engine)
Session

sessionmaker(class_='Session', bind=Engine(sqlite:///:memory:), autoflush=True, autocommit=False, expire_on_commit=True)

In [14]:
session = Session()
session

## Adding and Updating Objects

In [15]:
ed_user = User(name='ed', fullname='Ed Jones', nickname='edsnickname')
session.add(ed_user)
ed_user

<User(name = ed, fullname = Ed Jones, nickname = edsnickname)>

In [16]:
str(ed_user.id)

'None'

In [17]:
our_user = session.query(User).filter_by(name = 'ed').first()
our_user

<User(name = ed, fullname = Ed Jones, nickname = edsnickname)>

In [18]:
ed_user.id

1

In [19]:
our_user is ed_user

True

In [20]:
session.add_all([User(name='wendy', fullname='Wendy Williams', nickname='windy'),
                 User(name='mary', fullname='Mary Contrary', nickname='mary'),
                 User(name='fred', fullname='Fred Flintstone', nickname='freddy')])

In [21]:
ed_user.nickname = 'eddie'

In [22]:
session.dirty

IdentitySet([<User(name = ed, fullname = Ed Jones, nickname = eddie)>])

In [23]:
session.new

IdentitySet([<User(name = wendy, fullname = Wendy Williams, nickname = windy)>, <User(name = mary, fullname = Mary Contrary, nickname = mary)>, <User(name = fred, fullname = Fred Flintstone, nickname = freddy)>])

In [24]:
session.commit()

In [25]:
ed_user.id

1

## Rolling Back

In [26]:
ed_user.name = 'Edwardo'

In [27]:
fake_user = User(name='fakeuser', fullname='Invalid', nickname='12345')
session.add(fake_user)

In [28]:
session.query(User).filter(User.name.in_(['Edwardo', 'fakeuser'])).all()

[<User(name = Edwardo, fullname = Ed Jones, nickname = eddie)>,
 <User(name = fakeuser, fullname = Invalid, nickname = 12345)>]

In [29]:
ed_user.name

'Edwardo'

In [30]:
session.rollback()

In [31]:
ed_user.name

'ed'

In [32]:
fake_user in session

False

In [33]:
session.query(User).filter(User.name.in_(['Edwardo', 'fakeuser'])).all()

[]

## Querying

In [34]:
for instance in session.query(User).order_by(User.id):
    print(instance.name, instance.fullname)

ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flintstone


In [35]:
for name, fullname in session.query(User.name, User.fullname):
    print(name, fullname)

ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flintstone


In [36]:
for row in session.query(User, User.name).all():
    print(row.User, row.name)

<User(name = ed, fullname = Ed Jones, nickname = eddie)> ed
<User(name = wendy, fullname = Wendy Williams, nickname = windy)> wendy
<User(name = mary, fullname = Mary Contrary, nickname = mary)> mary
<User(name = fred, fullname = Fred Flintstone, nickname = freddy)> fred


In [37]:
for row in session.query(User.name.label('name_label')).all():
    print(row.name_label)

ed
wendy
mary
fred


In [38]:
from sqlalchemy.orm import aliased

user_alias = aliased(User, name='user_alias')

In [39]:
for row in session.query(user_alias, user_alias.name).all():
    print(row.user_alias)

<User(name = ed, fullname = Ed Jones, nickname = eddie)>
<User(name = wendy, fullname = Wendy Williams, nickname = windy)>
<User(name = mary, fullname = Mary Contrary, nickname = mary)>
<User(name = fred, fullname = Fred Flintstone, nickname = freddy)>


In [40]:
row.user_alias

<User(name = fred, fullname = Fred Flintstone, nickname = freddy)>

In [41]:
for u in session.query(User).order_by(User.id)[1:3]:
    print(u)

<User(name = wendy, fullname = Wendy Williams, nickname = windy)>
<User(name = mary, fullname = Mary Contrary, nickname = mary)>


In [42]:
for name, in session.query(User.name).filter_by(fullname ='Ed Jones'):
    print(name)

ed


In [43]:
for name, in session.query(User.name).filter(User.fullname=='Ed Jones'):
    print(name)

ed


In [44]:
for user in session.query(User).\
    filter(User.name=='ed').\
    filter(User.fullname=='Ed Jones'):
    print(user)

<User(name = ed, fullname = Ed Jones, nickname = eddie)>


## Common Filter Operators

In [45]:
list(session.query(User).filter(User.name == 'ed'))

[<User(name = ed, fullname = Ed Jones, nickname = eddie)>]

In [46]:
list(session.query(User).filter(User.name != 'ed'))

[<User(name = wendy, fullname = Wendy Williams, nickname = windy)>,
 <User(name = mary, fullname = Mary Contrary, nickname = mary)>,
 <User(name = fred, fullname = Fred Flintstone, nickname = freddy)>]

In [47]:
list(session.query(User).filter(User.name.like('%ed%')))

[<User(name = ed, fullname = Ed Jones, nickname = eddie)>,
 <User(name = fred, fullname = Fred Flintstone, nickname = freddy)>]

In [48]:
list(session.query(User).filter(User.name.ilike('%ed%')))

[<User(name = ed, fullname = Ed Jones, nickname = eddie)>,
 <User(name = fred, fullname = Fred Flintstone, nickname = freddy)>]

In [49]:
list(session.query(User).filter(User.name.in_(['ed', 'wendy', 'jack'])))

[<User(name = ed, fullname = Ed Jones, nickname = eddie)>,
 <User(name = wendy, fullname = Wendy Williams, nickname = windy)>]

In [50]:
list(session.query(User).filter(User.name.in_(
    session.query(User.name).filter(User.name.like('%ed%'))
)))

[<User(name = ed, fullname = Ed Jones, nickname = eddie)>,
 <User(name = fred, fullname = Fred Flintstone, nickname = freddy)>]

In [51]:
list(session.query(User).filter(~User.name.in_(['ed', 'wendy', 'jack'])))

[<User(name = mary, fullname = Mary Contrary, nickname = mary)>,
 <User(name = fred, fullname = Fred Flintstone, nickname = freddy)>]

In [52]:
list(session.query(User).filter(User.name == None))

[]

In [53]:
list(session.query(User).filter(User.name.is_(None)))

[]

In [54]:
list(session.query(User).filter(User.name != None))

[<User(name = ed, fullname = Ed Jones, nickname = eddie)>,
 <User(name = wendy, fullname = Wendy Williams, nickname = windy)>,
 <User(name = mary, fullname = Mary Contrary, nickname = mary)>,
 <User(name = fred, fullname = Fred Flintstone, nickname = freddy)>]

In [55]:
list(session.query(User).filter(User.name.isnot(None)))

[<User(name = ed, fullname = Ed Jones, nickname = eddie)>,
 <User(name = wendy, fullname = Wendy Williams, nickname = windy)>,
 <User(name = mary, fullname = Mary Contrary, nickname = mary)>,
 <User(name = fred, fullname = Fred Flintstone, nickname = freddy)>]

In [56]:
from sqlalchemy import and_

list(session.query(User).filter(and_(User.name == 'ed', 
                                     User.fullname == 'Ed Jones')))

[<User(name = ed, fullname = Ed Jones, nickname = eddie)>]

In [57]:
list(session.query(User).filter(User.name == 'ed', 
                                User.fullname == 'Ed Jones'))

[<User(name = ed, fullname = Ed Jones, nickname = eddie)>]

In [58]:
list(session.query(User).\
     filter(User.name == 'ed').\
     filter(User.fullname == 'Ed Jones'))

[<User(name = ed, fullname = Ed Jones, nickname = eddie)>]

In [59]:
from sqlalchemy import or_

list(session.query(User).filter(or_(User.name == 'ed', 
                                    User.name == 'wendy')))

[<User(name = ed, fullname = Ed Jones, nickname = eddie)>,
 <User(name = wendy, fullname = Wendy Williams, nickname = windy)>]

In [60]:
# list(session.query(User).filter(User.name.match('wendy')))

## Returning Lists and Scalars

In [61]:
query = session.query(User).filter(User.name.like('%ed')).order_by(User.id)
query

In [62]:
query.all()

[<User(name = ed, fullname = Ed Jones, nickname = eddie)>,
 <User(name = fred, fullname = Fred Flintstone, nickname = freddy)>]

In [63]:
query.first()

<User(name = ed, fullname = Ed Jones, nickname = eddie)>

In [64]:
# user = query.one()  # must be unique one

In [65]:
# user = query.filter(User.id == 99).one()

In [66]:
user = query.filter(User.id == 99).one_or_none()
user

In [67]:
query = session.query(User.id).filter(User.name == 'ed').order_by(User.id)
query.scalar()

1

## Using Textual SQL

In [68]:
from sqlalchemy import text

for user in session.query(User).\
    filter(text("id<224")).\
    order_by(text("id")).all():
    print(user.name)

ed
wendy
mary
fred


In [69]:
session.query(User).filter(text("id<:value and name=:name")).\
    params(value=224, name='fred').order_by(User.id).one()

<User(name = fred, fullname = Fred Flintstone, nickname = freddy)>

In [70]:
session.query(User).from_statement(text("SELECT * FROM users where name=:name")).\
    params(name='ed').all()

[<User(name = ed, fullname = Ed Jones, nickname = eddie)>]

In [71]:
tmt = text("SELECT name, id, fullname, nickname FROM users where name=:name")
tmt

In [72]:
stmt = tmt.columns(User.name, User.id, User.fullname, User.nickname)
stmt

<sqlalchemy.sql.selectable.TextAsFrom at 0x20cb6193e80; TextAsFrom object>

In [73]:
session.query(User).from_statement(stmt).params(name='ed').all()

[<User(name = ed, fullname = Ed Jones, nickname = eddie)>]

In [74]:
tmt = text("SELECT name, id FROM users where name=:name")
tmt

In [75]:
stmt = tmt.columns(User.name, User.id)
stmt

<sqlalchemy.sql.selectable.TextAsFrom at 0x20cb619ac88; TextAsFrom object>

In [76]:
session.query(User.id, User.name).\
    from_statement(stmt).params(name='ed').all()

[(1, 'ed')]

## Counting

In [77]:
session.query(User).filter(User.name.like('%ed')).count()

2

In [78]:
from sqlalchemy import func

session.query(User.name, func.count(User.name)).group_by(User.name).all() 

[('ed', 1), ('fred', 1), ('mary', 1), ('wendy', 1)]

In [79]:
session.query(func.count('*')).select_from(User).scalar()

4

In [80]:
session.query(User).count()

4

## Building a Relationship

In [81]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

In [82]:
class Address(Base):
    
    __tablename__ = 'addresses'    

    id = Column(Integer, primary_key = True)
    user_id = Column(Integer, ForeignKey('users.id'))  # user_id 必須是 users.id
    user = relationship('User', back_populates = 'addresses')  # Address.user 指到 User Table
    
    email_address = Column(String, nullable = False)
    
    def __repr__(self):
        return "<Address(email_address='%s')>" % self.email_address        

In [83]:
User.addresses = relationship('Address', 
                              order_by = Address.id,
                              back_populates = 'user')

In [84]:
Base.metadata.create_all(engine)

## Working with Related Objects

In [85]:
jack = User(name='jack', fullname='Jack Bean', nickname='gjffdd')
jack.addresses

[]

In [86]:
jack.addresses = [Address(email_address='jack@google.com'),
                  Address(email_address='j25@yahoo.com')]

In [87]:
jack.addresses

[<Address(email_address='jack@google.com')>,
 <Address(email_address='j25@yahoo.com')>]

In [88]:
jack.addresses[0]

<Address(email_address='jack@google.com')>

In [89]:
jack.addresses[0].user

<User(name = jack, fullname = Jack Bean, nickname = gjffdd)>

In [90]:
session.add(jack)
session.commit()

In [91]:
jack.id

5

In [92]:
jack.addresses

[<Address(email_address='jack@google.com')>,
 <Address(email_address='j25@yahoo.com')>]

In [93]:
jack.addresses[0].id

1

In [94]:
jack.addresses[0].user_id

5

In [95]:
jack.addresses[0].user

<User(name = jack, fullname = Jack Bean, nickname = gjffdd)>

## Querying with Joins

In [96]:
for u, a in session.query(User, Address).\
    filter(User.id==Address.user_id).\
    filter(Address.email_address=='jack@google.com').all():
        print(u)
        print(a)

<User(name = jack, fullname = Jack Bean, nickname = gjffdd)>
<Address(email_address='jack@google.com')>


In [97]:
session.query(User).\
    join(Address).\
    filter(Address.email_address=='jack@google.com').all()

[<User(name = jack, fullname = Jack Bean, nickname = gjffdd)>]

In [98]:
session.query(User).join(Address, User.id==Address.user_id).all()

[<User(name = jack, fullname = Jack Bean, nickname = gjffdd)>]

In [99]:
session.query(User).join(User.addresses).all()

[<User(name = jack, fullname = Jack Bean, nickname = gjffdd)>]

In [100]:
session.query(User).join(Address, User.addresses).all()

[<User(name = jack, fullname = Jack Bean, nickname = gjffdd)>]

In [101]:
session.query(User).join('addresses').all()

[<User(name = jack, fullname = Jack Bean, nickname = gjffdd)>]

In [102]:
session.query(User).outerjoin('addresses').all()

[<User(name = ed, fullname = Ed Jones, nickname = eddie)>,
 <User(name = wendy, fullname = Wendy Williams, nickname = windy)>,
 <User(name = mary, fullname = Mary Contrary, nickname = mary)>,
 <User(name = fred, fullname = Fred Flintstone, nickname = freddy)>,
 <User(name = jack, fullname = Jack Bean, nickname = gjffdd)>]

## Using Aliases

In [103]:
from sqlalchemy.orm import aliased

adalias1 = aliased(Address)
adalias2 = aliased(Address)

for username, email1, email2 in \
    session.query(User.name, adalias1.email_address, adalias2.email_address).\
    join(adalias1, User.addresses).\
    join(adalias2, User.addresses).\
    filter(adalias1.email_address=='jack@google.com').\
    filter(adalias2.email_address=='j25@yahoo.com'):
    
    print(username, email1, email2)

jack jack@google.com j25@yahoo.com


## Using Subqueries

In [104]:
from sqlalchemy.sql import func

stmt = session.query(Address.user_id, func.count('*').\
                     label('address_count')).\
                     group_by(Address.user_id).subquery()
stmt

<sqlalchemy.sql.selectable.Alias at 0x20cb60b93c8; %(2253617075144 anon)s>

In [105]:
for u, count in session.query(User, stmt.c.address_count).\
    outerjoin(stmt, User.id==stmt.c.user_id).order_by(User.id):
    
    print(u, count)

<User(name = ed, fullname = Ed Jones, nickname = eddie)> None
<User(name = wendy, fullname = Wendy Williams, nickname = windy)> None
<User(name = mary, fullname = Mary Contrary, nickname = mary)> None
<User(name = fred, fullname = Fred Flintstone, nickname = freddy)> None
<User(name = jack, fullname = Jack Bean, nickname = gjffdd)> 2


## Selecting Entities from Subqueries

In [106]:
stmt = session.query(Address).\
    filter(Address.email_address != 'j25@yahoo.com').\
    subquery()
stmt

<sqlalchemy.sql.selectable.Alias at 0x20cb6161f60; %(2253617766240 anon)s>

In [107]:
adalias = aliased(Address, stmt)
adalias

<AliasedClass at 0x20cb60a2048; Address>

In [108]:
for user, address in session.query(User, adalias).\
    join(adalias, User.addresses):
    
    print(user)
    print(address)

<User(name = jack, fullname = Jack Bean, nickname = gjffdd)>
<Address(email_address='jack@google.com')>


## Using EXISTS

In [109]:
from sqlalchemy.sql import exists

stmt = exists().where(Address.user_id==User.id)
stmt

In [110]:
for name, in session.query(User.name).filter(stmt):
    print(name)

jack


In [111]:
for name, in session.query(User.name).\
    filter(User.addresses.any()):
    
    print(name)

jack


In [112]:
for name, in session.query(User.name).\
    filter(User.addresses.any(Address.email_address.like('%google%'))):
    
    print(name)

jack


In [113]:
session.query(Address).\
    filter(~Address.user.has(User.name=='jack')).all() 

[]

## Common Relationship Operators

In [114]:
list(session.query(Address).filter(Address.user == jack))

[<Address(email_address='jack@google.com')>,
 <Address(email_address='j25@yahoo.com')>]

In [115]:
list(session.query(Address).filter(Address.user != jack))

[]

In [116]:
list(session.query(Address).filter(Address.user == None))

[]

In [117]:
list(session.query(User).filter(User.addresses.contains(jack.addresses[0])))

[<User(name = jack, fullname = Jack Bean, nickname = gjffdd)>]

In [118]:
list(session.query(User).filter(User.addresses.any(Address.email_address == 'jack@google.com')))

[<User(name = jack, fullname = Jack Bean, nickname = gjffdd)>]

In [119]:
list(session.query(Address).filter(Address.user.has(name = 'jack')))

[<Address(email_address='jack@google.com')>,
 <Address(email_address='j25@yahoo.com')>]

In [120]:
list(session.query(Address).with_parent(jack, 'addresses'))

[<Address(email_address='jack@google.com')>,
 <Address(email_address='j25@yahoo.com')>]

## Eager Loading
### Selectin Load

In [121]:
from sqlalchemy.orm import selectinload

jack = session.query(User).\
    options(selectinload(User.addresses)).\
    filter_by(name='jack').one()

jack

<User(name = jack, fullname = Jack Bean, nickname = gjffdd)>

In [122]:
jack.addresses

[<Address(email_address='jack@google.com')>,
 <Address(email_address='j25@yahoo.com')>]

### Joined Load

In [123]:
from sqlalchemy.orm import joinedload

jack = session.query(User).\
    options(joinedload(User.addresses)).\
    filter_by(name='jack').one()

jack

<User(name = jack, fullname = Jack Bean, nickname = gjffdd)>

In [124]:
jack.addresses

[<Address(email_address='jack@google.com')>,
 <Address(email_address='j25@yahoo.com')>]

### Explicit Join + Eagerload

In [125]:
from sqlalchemy.orm import contains_eager

jacks_addresses = session.query(Address).\
    join(Address.user).\
    filter(User.name=='jack').\
    options(contains_eager(Address.user)).all()

jacks_addresses

[<Address(email_address='jack@google.com')>,
 <Address(email_address='j25@yahoo.com')>]

In [126]:
jacks_addresses[0].user

<User(name = jack, fullname = Jack Bean, nickname = gjffdd)>

## Deleting

In [127]:
session.delete(jack)
session.query(User).filter_by(name='jack').count()

0

In [128]:
session.query(Address).filter(Address.email_address.in_(['jack@google.com', 
                                                         'j25@yahoo.com'])
                             ).count()

2

In [129]:
session.query(Address).filter(Address.email_address.in_(['jack@google.com', 
                                                         'j25@yahoo.com'])
                             ).count()

2

### Configuring delete/delete-orphan Cascade

In [130]:
session.close()

In [131]:
Base = declarative_base()

In [132]:
class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)

    addresses = relationship("Address", back_populates='user',
                    cascade="all, delete, delete-orphan")

    def __repr__(self):
        return "<User(name='%s', fullname='%s', nickname='%s')>" % (
            self.name, self.fullname, self.nickname)

In [133]:
class Address(Base):
    __tablename__ = 'addresses'
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'))
    user = relationship("User", back_populates="addresses")

def __repr__(self):
    return "<Address(email_address='%s')>" % self.email_address

In [134]:
jack = session.query(User).get(5)
jack

<User(name='jack', fullname='Jack Bean', nickname='gjffdd')>

In [135]:
del jack.addresses[1]

In [136]:
session.query(Address).filter(Address.email_address.in_(['jack@google.com', 
                                                         'j25@yahoo.com'])
                             ).count()

1

In [137]:
session.delete(jack)

In [138]:
session.query(User).filter_by(name='jack').count()

0

## Building a Many To Many Relationship

In [139]:
from sqlalchemy import Table, Text

# association table
post_keywords = Table('post_keywords', 
                      Base.metadata,
                      Column('post_id', ForeignKey('posts.id'), primary_key=True),
                      Column('keyword_id', ForeignKey('keywords.id'), primary_key=True))

In [140]:
class BlogPost(Base):
    __tablename__ = 'posts'
    
    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey('users.id'))
    headline = Column(String(255), nullable=False)
    body = Column(Text)

    # many to many BlogPost<->Keyword
    keywords = relationship('Keyword',
                            secondary=post_keywords,
                            back_populates='posts')

    def __init__(self, headline, body, author):
        self.author = author
        self.headline = headline
        self.body = body

    def __repr__(self):
        return "BlogPost(%r, %r, %r)" % (self.headline, self.body, self.author)


class Keyword(Base):
    __tablename__ = 'keywords'

    id = Column(Integer, primary_key=True)
    keyword = Column(String(50), nullable=False, unique=True)
    posts = relationship('BlogPost',
                         secondary=post_keywords,
                         back_populates='keywords')

    def __init__(self, keyword):
        self.keyword = keyword

In [141]:
BlogPost.author = relationship(User, back_populates="posts")
User.posts = relationship(BlogPost, back_populates="author", lazy="dynamic")

In [142]:
Base.metadata.create_all(engine)

In [143]:
wendy = session.query(User).filter_by(name='wendy').one()
wendy

<User(name='wendy', fullname='Wendy Williams', nickname='windy')>

In [144]:
post = BlogPost("Wendy's Blog Post", "This is a test", wendy)
session.add(post)

post

BlogPost("Wendy's Blog Post", 'This is a test', <User(name='wendy', fullname='Wendy Williams', nickname='windy')>)

In [145]:
post.keywords.append(Keyword('wendy'))
post.keywords.append(Keyword('firstpost'))

In [146]:
session.query(BlogPost).\
    filter(BlogPost.keywords.any(keyword='firstpost')).\
    all()

[BlogPost("Wendy's Blog Post", 'This is a test', <User(name='wendy', fullname='Wendy Williams', nickname='windy')>)]

In [147]:
session.query(BlogPost).\
    filter(BlogPost.author==wendy).\
    filter(BlogPost.keywords.any(keyword='firstpost')).\
    all()

[BlogPost("Wendy's Blog Post", 'This is a test', <User(name='wendy', fullname='Wendy Williams', nickname='windy')>)]

In [148]:
wendy.posts.\
filter(BlogPost.keywords.any(keyword='firstpost')).\
all()

[BlogPost("Wendy's Blog Post", 'This is a test', <User(name='wendy', fullname='Wendy Williams', nickname='windy')>)]